* rerun simulation high quality simulation for optimization trajectories
* establish theoretical justification for Degeneracy and Insensitivity

In [179]:
import pickle,scqubits
from pandas import concat
from numpy import pi,log10,isinf,isnan,linspace
from scipy.stats import pearsonr
from DiSuQ.utils import plotCompare,plotScatter

### data loading

In [2]:
def aggregate(Sim):
    Agg = []
    for index,sample in enumerate(Sim):
        metric = sample[0]
        path = sample[2].iloc[1:].reset_index()
        sample = concat([metric,path],axis=1,join='inner').drop(['index'],axis=1)
        sample['sample'] = index
        Agg.append(sample)
    Agg = concat(Agg).reset_index()
    Agg['delta'] *= -1
    #Agg['loc'] = Agg['sample'].apply(lambda circ:location[circ])
    Agg['EcS'] = 1/(1/Agg['Cx'] + 1/Agg['CJx'])
    return Agg

In [3]:
with open(r"./indPiDegeneracy.pickle", "rb") as input_file:
    PiDeg = aggregate(pickle.load(input_file))

In [4]:
with open(r"./indPiInsensitivity.pickle", "rb") as input_file:
    PiIns = aggregate(pickle.load(input_file))

In [5]:
with open(r"./simPi.pickle", "rb") as input_file:
    PiSim = aggregate(pickle.load(input_file))

In [6]:
with open(r"./indZeroDegeneracy.pickle", "rb") as input_file:
    ZeroDeg = aggregate(pickle.load(input_file))

In [7]:
with open(r"./indZeroInsensitivity.pickle", "rb") as input_file:
    ZeroIns = aggregate(pickle.load(input_file))

In [8]:
with open(r"./indZeroInsensitivity.pickle", "rb") as input_file:
    ZeroIns = aggregate(pickle.load(input_file))

In [9]:
with open(r"./simZero.pickle", "rb") as input_file:
    ZeroSim = aggregate(pickle.load(input_file))

In [10]:
with open(r"./simZeroThird.pickle", "rb") as input_file:
    ZeroThird = aggregate(pickle.load(input_file))

In [11]:
with open(r"./simZeroSub.pickle", "rb") as input_file:
    ZeroSub = aggregate(pickle.load(input_file))

In [12]:
with open(r"./simZeroSubIns.pickle", "rb") as input_file:
    ZeroSubIns = aggregate(pickle.load(input_file))

### sc Qubit simulation

In [13]:
def representationCapacity(El,Ej,Ec,EcJ):
    basis = {'Chi':7,'Theta':64,'Phi':1024}
    # theta capacity is sufficient for the given parameter range
    basis = {'Chi':7,'Theta':12,'Phi':1024}
    flux0 = 24*pi*2
    return flux0,basis

In [14]:
def HQSC(Agg,op=0):
    L = len(Agg)
    Degeneracy,Delta,E10,E20 = [],[],[],[]
    for index,circ in Agg.iterrows():
        El = circ['Lx']; Ej = circ['Jx']; EcJ = circ['CJx']; Ec = circ['Cx']
        EcS = 1/(1/Ec+1/EcJ)
        flux0,basis = representationCapacity(El,Ej,Ec,EcJ)
        flux_grid = scqubits.Grid1d(-flux0, flux0, basis['Phi'])

        zero_pi = scqubits.FullZeroPi(grid = flux_grid,
                           EJ   = Ej,
                           EL   = El,
                           ECJ  = EcJ, EC = None, ECS = EcS,
                           #EC  = 1/(1/EcS-1/EcJ),
                           ng   = 0, dEJ  = 0.0,
                             dCJ  = 0.0,
                             dEL  = 0.0,
                             dC   = 0.0,
                           flux = op,
                           ncut = basis['Theta'], zeropi_cutoff = 5,
                             zeta_cutoff = basis['Chi'])
        e0,e1,e2 = zero_pi.eigenvals(3)
        zero_pi.flux = op + .05
        n0,n1,n2 = zero_pi.eigenvals(3)
        D = log10((e2-e0)/(e1-e0))
        delta = log10((e1-e0)/abs((e1-e0)-(n1-n0)))
        Degeneracy.append(D); Delta.append(delta); E10.append(e1-e0); E20.append(e2-e0)
        if index%50 == 0:
            print(index,'/',L)
    Agg['D*'] = Degeneracy ; Agg['delta*'] = Delta ; Agg['E10*'] = E10 ; Agg['E20*'] = E20
    return Agg

### recalculation

In [15]:
PiIns = HQSC(PiIns,.5)
PiDeg = HQSC(PiDeg,.5)
PiSim = HQSC(PiSim,.5)
ZeroDeg = HQSC(ZeroDeg,0)
ZeroIns = HQSC(ZeroIns,0)
ZeroSim = HQSC(ZeroSim,0)

ZeroSub = HQSC(ZeroSub,0)
ZeroSubIns = HQSC(ZeroSubIns,0)
ZeroThird = HQSC(ZeroThird,0)

0 / 220
50 / 220


/tmp/ipykernel_2574644/904122182.py:26: RuntimeWarning:

divide by zero encountered in scalar divide



100 / 220
150 / 220
200 / 220
0 / 190
50 / 190
100 / 190
150 / 190
0 / 178
50 / 178
100 / 178


/tmp/ipykernel_2574644/904122182.py:26: RuntimeWarning:

divide by zero encountered in scalar divide



150 / 178
0 / 244
50 / 244
100 / 244
150 / 244
200 / 244
0 / 240
50 / 240
100 / 240
150 / 240
200 / 240


/tmp/ipykernel_2574644/904122182.py:26: RuntimeWarning:

divide by zero encountered in scalar divide



0 / 276


/tmp/ipykernel_2574644/904122182.py:26: RuntimeWarning:

divide by zero encountered in scalar divide



50 / 276
100 / 276
150 / 276
200 / 276
250 / 276
0 / 43
0 / 38


/tmp/ipykernel_2574644/904122182.py:26: RuntimeWarning:

divide by zero encountered in scalar divide



In [162]:
ZeroThird = HQSC(ZeroThird,0)

0 / 244
50 / 244
100 / 244
150 / 244
200 / 244


/tmp/ipykernel_2574644/904122182.py:26: RuntimeWarning:

divide by zero encountered in scalar divide



In [108]:
Pi = concat([PiDeg,PiIns,PiSim])

In [109]:
Zero = concat([ZeroDeg,ZeroIns,ZeroSim])

### ratio-metric correlation plots

In [119]:
Agg = Pi ; print(len(Agg))

588


In [125]:
x = log10(Agg['CJx']/Agg['EcS'])
x = log10(Agg['Jx']/Agg['EcS'])
# x = log10(Agg['CJx']/Agg['Lx'])
# x = log10(Agg['Jx']/Agg['Lx'])
#y = Agg['D*'] ; Y = Agg['D']
y = Agg['delta*'] ; Y = Agg['delta']
r,p = pearsonr(x[~isinf(y)],y[~isinf(y)])
R,P = pearsonr(x[~isinf(Y)],Y[~isinf(Y)])

print(R,r)

-0.8410153610179373 -0.8368515030574436


In [107]:
plotScatter(log10(Agg['CJx']/Agg['EcS']),{'D':Agg['D'],'Delta':Agg['delta']},x_label='log(EcJ/EcS)',dot_size=5,
            size=(500,500),legend=True)
plotScatter(log10(Agg['Jx']/Agg['EcS']),{'D':Agg['D*'],'Delta':Agg['delta*']},x_label='log(Ej/EcS)',dot_size=5,
            size=(500,500),legend=True)
plotScatter(log10(Agg['CJx']/Agg['Lx']),{'D':Agg['D*'],'Delta':Agg['delta*']},x_label='log(EcJ/El)',dot_size=5,
            size=(500,500),legend=True)
plotScatter(log10(Agg['Jx']/Agg['Lx']),{'D':Agg['D*'],'Delta':Agg['delta*']},x_label='log(Ej/El)',dot_size=5,
            size=(500,500),legend=True)

## Superinductance

In [138]:
Agg = Pi
final = Agg.groupby('sample').nth(-1)

In [140]:
plotScatter(log10(final['Lx']),{'delta':final['delta*'],'D':final['D*']},x_label='El',dot_size=10,size=(500,500))

In [137]:
plotScatter(Agg['Lx'],{'D*':Agg['D*']},x_label='El',y_label='degeneracy',dot_size=10,size=(500,500))

### mapping Low quality to High quality

In [193]:
Agg = ZeroThird

In [194]:
plotScatter(Agg['D'],{'HQ':Agg['D*']},x_label='D',y_label='D*',dot_size=2.5,size=(500,500))

In [165]:
plotScatter(Agg['delta'],{'HQ':Agg['delta*']},x_label='delta',y_label='delta*',dot_size=2.5,size=(500,500))

## optimal circuits

In [196]:
Agg = Pi
Agg.loc[(Agg['D*']>.95) & (Agg['delta*']>2)]

,index,delta,D,E10,E20,loss,time,Jx,Jy,CJx,...,Lx,Ly,Cx,Cy,sample,EcS,D*,delta*,E10*,E20*
89,1,-2.068186,2.049218,0.000015,0.001709,2.068186,148.713088,100.187965,100.187965,99.980049,...,0.000003,0.000003,0.113300,0.113300,27,0.113172,2.224701,2.346340,0.00001,0.001721
90,2,-1.560305,1.583577,0.000046,0.001755,1.560305,217.561675,100.187408,100.187408,99.980064,...,0.000003,0.000003,0.113914,0.113914,27,0.113784,2.233362,2.331184,0.00001,0.001726
91,3,-1.560305,1.583577,0.000046,0.001755,1.560305,389.186248,100.187408,100.187408,99.980064,...,0.000003,0.000003,0.113914,0.113914,27,0.113784,2.233362,2.331184,0.00001,0.001726


In [189]:
Agg = Zero
Agg.loc[(Agg['D*']>1) & (Agg['delta*']>0)]

,index,delta,D,E10,E20,loss,time,Jx,Jy,CJx,...,Lx,Ly,Cx,Cy,sample,EcS,D*,delta*,E10*,E20*
24,0,4.244743,0.300041,0.201059,0.401204,-0.757464,NaN,9.990625,9.990625,99.999992,...,0.163465,0.163465,0.030763,0.030763,7,0.030753,1.493849,1.921213,0.006433,0.200571
25,1,5.323557,0.300254,0.200890,0.401062,-0.937302,1126.767822,9.990625,9.990625,99.999992,...,0.163465,0.163465,0.030762,0.030762,7,0.030753,1.493848,1.921213,0.006433,0.200571
26,2,5.323557,0.300254,0.200890,0.401062,-0.937302,572.989715,9.990625,9.990625,99.999992,...,0.163465,0.163465,0.030762,0.030762,7,0.030753,1.493848,1.921213,0.006433,0.200571


In [192]:
Agg = ZeroThird
Agg.loc[(Agg['D*']>.95) & (Agg['delta*']>-10)]

,index,delta,D,E10,E20,loss,time,Jx,Jy,CJx,...,Lx,Ly,Cx,Cy,sample,EcS,D*,delta*,E10*,E20*
24,0,3.468435,0.300872,0.274828,0.549456,-1.356726,NaN,9.967607,9.967607,99.999992,...,0.163465,0.163465,0.057701,0.057701,7,0.057667,1.344165,2.529560,0.012436,0.274693
28,0,4.009536,0.299796,0.311951,0.622131,-1.536376,NaN,99.998863,99.998863,193.702484,...,0.163465,0.163465,0.074053,0.074053,8,0.074024,1.265594,1.909675,0.016882,0.311191
101,0,3.908332,0.301095,0.864868,1.729996,-1.503507,NaN,100.001442,100.001442,193.702484,...,0.163465,0.163465,0.571961,0.571961,24,0.570277,1.752921,1.909946,0.015276,0.864849
102,1,4.054529,0.301003,0.865005,1.729904,-1.552178,113.536734,100.001442,100.001442,193.702484,...,0.163465,0.163465,0.572003,0.572003,24,0.570319,1.752940,1.909947,0.015276,0.864881
103,2,4.054529,0.301003,0.865005,1.729904,-1.552178,646.285522,100.001442,100.001442,193.702484,...,0.163465,0.163465,0.572003,0.572003,24,0.570319,1.752940,1.909947,0.015276,0.864881
105,1,3.698241,0.300976,0.990158,1.980072,-1.433398,81.885856,100.000061,100.000061,193.702484,...,0.163465,0.163465,0.749589,0.749589,25,0.746699,1.818331,1.917634,0.015043,0.990076
106,2,3.698241,0.300976,0.990158,1.980072,-1.433398,510.335000,100.000061,100.000061,193.702484,...,0.163465,0.163465,0.749589,0.749589,25,0.746699,1.818331,1.917634,0.015043,0.990076


#### spectrum

In [184]:
from DiSuQ.Torch import models
import importlib;importlib.reload(models)

<module 'DiSuQ.Torch.models' from '/Users/chishti/JupyterHub/DiSuQ/Torch/models.py'>

In [201]:
circuit = ZeroThird.iloc[179]

In [202]:
El,Ec,Ej,EcJ = circuit['Lx'],circuit['Cx'],circuit['Jx'],circuit['CJx']
flux0,basis = representationCapacity(El,Ec,Ej,EcJ)
flux0 *= 5
basis['Phi'] *= 5
zero_pi = models.SCzeroPi(basis,flux0,Ej,Ec,El,EcJ)

In [203]:
flux_range = linspace(0,1,5001,endpoint=True)
spectrum = zero_pi.get_spectrum_vs_paramvals('flux',flux_range,evals_count=3)
G,I,II = spectrum.energy_table.T

Spectral data:   0%|          | 0/5001 [00:00<?, ?it/s]

In [207]:
# 179 # SUper High Resolution
plotCompare(flux_range,{'E10':I-G,'E20':II-G},
            x_label=r'$$\Phi_{ext}$$',y_label='excitation(GHz)',size=(500,700))

In [200]:
# 179
plotCompare(flux_range,{'E10':I-G,'E20':II-G},
            x_label=r'$$\Phi_{ext}$$',y_label='excitation(GHz)',size=(500,600))

In [210]:
Pi.iloc[24]

index     1.000000e+00
delta    -4.540542e+00
D         4.540273e+00
E10       3.051758e-05
E20       1.058823e+00
loss     -4.540273e+00
time      4.856951e+02
Jx        1.000802e+02
Jy        1.000802e+02
CJx       5.000209e-01
CJy       5.000209e-01
Lx        6.373561e-01
Ly        6.373561e-01
Cx        2.198326e-01
Cy        2.198326e-01
sample    9.000000e+00
EcS       1.526990e-01
D*        1.242890e+01
delta*   -1.220051e+01
E10*      3.943512e-13
E20*      1.058722e+00
Name: 24, dtype: float64

In [188]:
# 24 ZeroThird.iloc[24]
plotCompare(flux_range,{'E10':I-G,'E20':II-G},
            x_label=r'$$\Phi_{ext}$$',y_label='excitation(GHz)',size=(500,600))

## learning Zero-Pi degeneracy/insensitivity manifold
* optimization trajectories are geodesics over the manifold
$$(El,Ec,Ej,EcJ) \equiv R_+^4 \rightarrow R_+ \otimes R \equiv (D,\Delta)$$
* data points are derived from optimization trajectories
* Learning incorporate entanglement and quantum physics of Zero-Pi 